In [53]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [54]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
import requests

In [55]:
import sys
print(sys.path)
sys.path.insert(0, "/Users/garciaj42/code/MLFinance/claude")

['/Users/garciaj42/code/MLFinance/claude', '/Users/garciaj42/code/MLFinance/claude/notebooks', '/opt/homebrew/Cellar/python@3.9/3.9.19_1/Frameworks/Python.framework/Versions/3.9/lib/python39.zip', '/opt/homebrew/Cellar/python@3.9/3.9.19_1/Frameworks/Python.framework/Versions/3.9/lib/python3.9', '/opt/homebrew/Cellar/python@3.9/3.9.19_1/Frameworks/Python.framework/Versions/3.9/lib/python3.9/lib-dynload', '', '/opt/homebrew/lib/python3.9/site-packages', '/opt/homebrew/lib/python3.9/site-packages/IPython/extensions', '/Users/garciaj42/.ipython']


In [56]:
lookback_days = 365 * 1

In [57]:
def generate_summary_stats(df):
    """
    Generate summary statistics for the dataset.
    
    Parameters:
    df (pandas.DataFrame): Input DataFrame
    
    Returns:
    pandas.DataFrame: Summary statistics
    """
    stats = pd.DataFrame({
        'start_date': df.index.min(),
        'end_date': df.index.max(),
        'data_points': df.count(),
        'missing_pct': (df.isna().sum() / len(df) * 100).round(2),
        'mean': df.mean().round(4),
        'std': df.std().round(4),
        'min': df.min().round(4),
        'max': df.max().round(4)
    }).T
    
    return stats

In [58]:
from market_indicator_data import fetch_market_indicators, fill_missing_data

# Fetch the data
market_indicators_data = fetch_market_indicators(datetime.now() -  timedelta(days=lookback_days))

Successfully fetched data for Gold (GC=F)
Successfully fetched data for USD/JPY (JPY=X)
Successfully fetched data for S&P 500 (^GSPC)
Successfully fetched data for VIX (^VIX)
Successfully fetched data for ASX 200 (Australia) (^AXJO)
Successfully fetched data for FTSE 100 (UK) (^FTSE)
Successfully fetched data for DAX (Germany) (^GDAXI)
Successfully fetched data for EUR/USD (EUR=X)
Successfully fetched data for Crude Oil (CL=F)


$^NYHILO: possibly delisted; no timezone found


Successfully fetched data for S&P 500 ETF (SPY)
No data available for NYSE New High/Low Index (^NYHILO)
Successfully fetched data for 20+ Year Treasury Bond ETF (TLT)
Successfully fetched data for Investment Grade Corporate Bond ETF (LQD)
Successfully fetched data for Copper (HG=F)


In [59]:
market_indicators_data

,GC=F,JPY=X,^GSPC,^VIX,^AXJO,^FTSE,^GDAXI,EUR=X,CL=F,SPY,TLT,LQD,HG=F
Date,,,,,,,,,,,,,
2023-12-25 00:00:00+00:00,NaN,142.341995,NaN,NaN,NaN,NaN,NaN,0.90690,NaN,NaN,NaN,NaN,NaN
2023-12-26 00:00:00+00:00,NaN,142.229996,NaN,NaN,NaN,NaN,NaN,0.90742,NaN,NaN,NaN,NaN,NaN
2023-12-26 05:00:00+00:00,2058.199951,NaN,4774.750000,NaN,NaN,NaN,NaN,NaN,75.570000,469.625916,94.863464,105.614868,3.8955
2023-12-26 06:00:00+00:00,NaN,NaN,NaN,12.990000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-12-26 13:00:00+00:00,NaN,NaN,NaN,NaN,7561.200195,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-19 06:00:00+00:00,NaN,NaN,NaN,24.090000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-12-19 23:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,19884.75,NaN,NaN,NaN,NaN,NaN,NaN
2024-12-20 00:00:00+00:00,NaN,157.643997,NaN,NaN,NaN,8084.600098,NaN,0.96479,NaN,NaN,NaN,NaN,NaN


In [60]:
# Generate and print summary statistics
summary_stats = generate_summary_stats(market_indicators_data)
print("\nSummary Statistics:")
print(summary_stats)
    
print("\nColumns in market indicators dataset:")
for col in market_indicators_data.columns:
    print(f"- {col}")


Summary Statistics:
                                  GC=F                      JPY=X  \
start_date   2023-12-25 00:00:00+00:00  2023-12-25 00:00:00+00:00   
end_date     2024-12-20 06:00:00+00:00  2024-12-20 06:00:00+00:00   
data_points                        250                        260   
missing_pct                      80.24                      79.45   
mean                         2379.3848                   151.0693   
std                           225.3726                     5.1156   
min                             1990.3                     140.79   
max                             2788.5                    161.621   

                                 ^GSPC                       ^VIX  \
start_date   2023-12-25 00:00:00+00:00  2023-12-25 00:00:00+00:00   
end_date     2024-12-20 06:00:00+00:00  2024-12-20 06:00:00+00:00   
data_points                        250                        250   
missing_pct                      80.24                      80.24   
mean        